In [2]:
import json
import numpy as np
import pandas as pd
import jieba
import progressbar

In [3]:
jieba.set_dictionary('merged_dict.txt')
jieba.initialize()

Building prefix dict from C:\Users\wormw\Documents\comp4641-project\nlp\merged_dict.txt ...
Loading model from cache C:\Users\wormw\AppData\Local\Temp\jieba.ub49ffea3258cac9781f19edeac8a403e.cache
Loading model cost 3.698 seconds.
Prefix dict has been built successfully.


In [4]:
# with open('../scraper/posts.json') as f:
#   posts = json.load(f)

posts_data = pd.read_pickle("../lihkg_data/li_data_final.pkl")

In [5]:
KNOWN_TROLLS = ['41853','194398','76776','71393','30019','37596','63950','310982','282494','72457','326966','322121','322637','121095','169','213597','226126','245201','159448','149494','181778','324892','273582','118226','240250','316329','77788','219767','71341','12186','228352','75196','28435','25002','247732','322304','258598','133525','14941','27416','222907','48631','158008','289946','96230','40470','186800','30788','288523','254191','84585','85242','123870','89514','170281','103799','64699','49699','299923','71140','122676','61748','202464','288351','234658','277044','149978','6237','98232','69484','165028','328913','132380','146030','331780','266100','52773','275585','249366','41351','273474','39751','51615','326708','213952','1210','270563','241151','70493','335209','276916','232237','94819','78468','203248','100028','291185','95649','332154','83060','264328','244624','105890','5591','171664','23721','129619','326414','14965','51985','191271','253253','25771','153956','32583','72825','336250','58089','70734','124277','173333','155597','137604','276602','41818','162674','151378','295632','147880','314774','57035','88029','97104','216114','64295','28833','203010','264400','273157','238973','57405','28870','132128','199261','126609','65530','172349','173849','192248','261633','127675','34757','166655','228589','343059','144849']

TROLLS_FROM_LABLING = ['71341', '119756', '268846', '161758', '324560', '103312', '12444', '101824', '294199', '134739', '198278']

RIBBISH_POST = ['推', 'Push', 'push', '1', '此回覆已被刪除', 'pish', 'PUSH']


In [6]:
test_train_ratio = 0.7  # Ratio for training set and testing set
resampling_ratio = 0.3  # Ratio for trolls and non-trolls
over_ratio = 0.7  # Ratio of oversampling and undersampling; ie. add vs remove

In [7]:
num_lines = posts_data.shape[0]
num_train = round(num_lines * test_train_ratio)
num_test = num_lines - num_train
print(num_lines)

7619975


In [ ]:
num_troll = 0
num_normal = 0
num_troll_train = 0
num_normal_train = 0
num_troll_test = 0
num_normal_test = 0

In [ ]:
train_lines = []
test_lines = []

In [ ]:
# for post in posts:
#     if len(post['msg'].strip()) != 0 and post['msg'] not in RIBBISH_POST:
#         if post['user_id'] in KNOWN_TROLLS:
#             lines.append('__label__TROLL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
#         else:
#             lines.append('__label__NORMAL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
#
# np.random.shuffle(lines)

posts_data = posts_data.sample(frac=1).reset_index(drop=True)

for index, post in posts_data.iterrows():
    if len(post['msg'].strip()) != 0 and post['msg'] not in RIBBISH_POST:
        if (post['user_id'] in KNOWN_TROLLS) or (post['user_id'] in TROLLS_FROM_LABLING):
            if index <= num_train:
                train_lines.append('__label__TROLL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
                num_troll_train += 1
            else:
                test_lines.append('__label__TROLL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
                num_troll_test += 1
            num_troll += 1
        else:
            if index <= num_train:
                train_lines.append('__label__NORMAL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
                num_normal_train += 1
            else:
                test_lines.append('__label__NORMAL ' + ' '.join(jieba.cut(post['msg'].replace('\n',' ').replace(' ', '').strip(), cut_all=False)))
                num_normal_test += 1
            num_normal += 1



In [ ]:
print("Statistics:")
print("Total number of lines:", num_lines)
print("Total number of lines in training:", num_train)
print("Total number of lines in testing:", num_test)
print("Total number of troll lines:", num_troll)
print("Total number of non-troll lines:", num_normal)
print("Total number of troll lines in training:", num_troll_train)
print("Total number of non-troll lines in training:", num_normal_train)
print("Total number of troll lines in testing:", num_troll_test)
print("Total number of non-troll lines in testing:", num_normal_test)

In [ ]:
num_trolls_missing = round(num_training * resampling_ratio) - num_troll_train
if num_trolls_missing > 0:
    num_trolls_add = round(num_trolls_missing * over_ratio)
    num_normal_remove = num_trolls_missing - num_trolls_add
    while num_normal_remove > 0:
        rand_index = np.random.randint(0, len(train_lines))
        if train_lines[rand_index][0:15] == '__label__NORMAL':
            train_lines.pop(rand_index)
            num_normal_remove -= 1
    new_num_training = len(train_lines)
    while num_trolls_add > 0:
        rand_index = np.random.randint(0, new_num_training)
        if train_lines[rand_index][0:15] == '__label__TROLL ':
            train_lines.insert(len(train_lines), training[rand_index])
            num_trolls_add -= 1

In [ ]:
output = ''

for line in train_lines:
    output += line + '\n'

with open('../lihkg_data/nlp_train.txt', 'w') as the_file:
    the_file.write(output)

In [ ]:
output = ''

for line in test_lines:
    output += line + '\n'
            
with open('../lihkg_data/nlp_test.txt', 'w') as the_file:
    the_file.write(output)